In [1]:
import dill
dill.load_session('ref_reports.db')

In [2]:
import pandas as pd 
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


In [1]:
path = 'C:/Users/isaac/Dropbox/Apps/ShareLaTeX/information_lawyer_quality'

# Read data 
data = pd.read_csv(path + '/_aux/hd_data.csv') 
#data = data.sample(frac=0.1, replace=True, random_state=1)
print(data.shape)
data.columns

NameError: name 'pd' is not defined

In [4]:
data.describe()

,reclutamiento,giro_empresa_d2,giro_empresa_d3,giro_empresa_d4,giro_empresa_d5,giro_empresa_d6,giro_empresa_d7,giro_empresa_d8,giro_empresa_d9,giro_empresa_d10,...,min_vac,min_prima_vac,min_ley,c_sal_caidos,prop_hextra,modo_termino,liq_total,liq_total_tope,liq_total_pos,duracion
count,4997.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,...,4871.000000,4871.000000,4.998000e+03,4.123000e+03,4984.000000,4998.000000,4.998000e+03,4.998000e+03,4998.000000,4998.000000
mean,0.192516,0.000400,0.000400,0.002601,0.034014,0.003601,0.007003,0.052621,0.232093,0.022809,...,1938.003057,484.500675,5.269175e+04,2.294548e+05,0.369944,1.605842,2.854584e+04,2.761328e+04,0.653661,1.021339
std,0.394315,0.020002,0.020002,0.050939,0.181282,0.059910,0.083398,0.223298,0.422210,0.149310,...,6396.703136,1599.175766,1.089357e+05,5.586934e+05,0.314853,0.933609,1.400880e+05,1.275581e+05,0.475850,0.958674
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.990000,0.250000,0.000000e+00,1.394670e+03,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.005479
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,352.628555,88.155888,2.044597e+04,7.340045e+04,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.331507
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,848.219970,212.050000,3.172866e+04,1.178750e+05,0.394096,1.000000,8.000000e+03,8.000000e+03,1.000000,0.697260
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1867.498600,466.874645,5.436342e+04,2.186237e+05,0.641515,2.000000,2.073375e+04,2.073375e+04,1.000000,1.400000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,213154.980000,53288.746000,4.449881e+06,2.005620e+07,0.996460,4.000000,6.700000e+06,5.600418e+06,1.000000,4.960000


In [78]:
X = data[['reclutamiento',
'trabajador_base',
'gen',
'tipo_jornada_d2',
'tipo_jornada_d3',
'tipo_jornada_d4',
'reinst',
'indem',
'sal_caidos',
'prima_antig',
'prima_vac',
'hextra',
'rec20',
'prima_dom',
'desc_sem',
'desc_ob',
'sarimssinf',
'utilidades',
'monto_recsueldo',
'nulidad',
'vac',
'ag',
'codem',
'horas_extra',
'antiguedad',
'salario_diario',
'horas_sem',
'c_antiguedad',
'c_indem',
'c_prima_antig',
'c_rec20',
'c_ag',
'c_vac',
'c_hextra',
'c_prima_vac',
'c_prima_dom',
'c_desc_sem',
'c_desc_ob',
'c_utilidades',
'c_recsueldo',
'c_total',
'min_indem',
'min_prima_antig',
'min_ag',
'min_vac',
'min_prima_vac',
'min_ley',
'c_sal_caidos',
'prop_hextra'
]]


sett = data['modo_termino']==1
sett.astype('int64')
drp = data['modo_termino']==2
drp.astype('int64')
exp = data['modo_termino']==4
exp.astype('int64')
cr_win = (data['modo_termino']==3) & (data['liq_total_pos']==1)
cr_win.astype('int64')
cr_los = (data['modo_termino']==3) & (data['liq_total_pos']==0)
cr_los.astype('int64')

X_3 = X[data['modo_termino']==3] 
rslt_df = data.loc[data['modo_termino']==3]
cr_win_ph2 = rslt_df['liq_total_pos']==1
cr_win_ph2.astype('int64')
tot_comp_3 = rslt_df['liq_total_tope']
tot_comp_3l = np.log(rslt_df['liq_total_tope']+1)

X_31 = X[(data['modo_termino']==3) & (data['liq_total_pos']==1)] 
rslt_df = data.loc[(data['modo_termino']==3) & (data['liq_total_pos']==1)]
dur_31 = rslt_df['duracion']

X_30 = X[(data['modo_termino']==3) & (data['liq_total_pos']==0)] 
rslt_df = data.loc[(data['modo_termino']==3) & (data['liq_total_pos']==0)]
dur_30 = rslt_df['duracion']

X_1 = X[data['modo_termino']==1]
rslt_df = data.loc[data['modo_termino']==1]
tot_comp_1l = np.log(rslt_df['liq_total_tope']+1)
dur_1 = rslt_df['duracion']

X_2 = X[data['modo_termino']==2]
rslt_df = data.loc[data['modo_termino']==2]
dur_2l = np.log(rslt_df['duracion']+1)

X_4 = X[data['modo_termino']==4]
rslt_df = data.loc[data['modo_termino']==4]
dur_4 = rslt_df['duracion']

In [79]:
numeric_featuresX = ['antiguedad',
'salario_diario',
'horas_sem',
'c_antiguedad',
'c_indem',
'c_prima_antig',
'c_rec20',
'c_ag',
'c_vac',
'c_hextra',
'c_prima_vac',
'c_prima_dom',
'c_desc_sem',
'c_desc_ob',
'c_utilidades',
'c_recsueldo',
'c_total',
'min_indem',
'min_prima_antig',
'min_ag',
'min_vac',
'min_prima_vac',
'min_ley',
'c_sal_caidos',
'prop_hextra']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())])




binary_featuresX = ['reclutamiento',
'trabajador_base',
'gen',
'tipo_jornada_d2',
'tipo_jornada_d3',
'tipo_jornada_d4',
'reinst',
'indem',
'sal_caidos',
'prima_antig',
'prima_vac',
'hextra',
'rec20',
'prima_dom',
'desc_sem',
'desc_ob',
'sarimssinf',
'utilidades',
'monto_recsueldo',
'nulidad',
'vac',
'ag',
'codem',
'horas_extra']

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=1, include_bias=False))])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresX),
        ('bin', binary_transformer, binary_featuresX)])


In [83]:
from sklearn.model_selection import train_test_split

[X_train, X_test, sett_train, sett_test, cr_los_train, cr_los_test,
 cr_win_train, cr_win_test, exp_train, exp_test, 
 drp_train, drp_test] = train_test_split(X, sett, cr_los, cr_win, exp, drp, 
                                                                  test_size=0.2, random_state=0)

[X_3_train, X_3_test, cr_win_ph2_train, cr_win_ph2_test] = train_test_split( X_3, cr_win_ph2, 
                                                                  test_size=0.2, random_state=0)

In [86]:
from sklearn.linear_model import LogisticRegression
# Hyperparameters - grid
# ==============================================================================
param_grid = {'model__penalty'  : ['l1', 'l2', 'elasticnet'],
              'model__C'     : [0.0001, 0.001, 0.01, 0.1, 1, 10, 20]
             }


logit_est = LogisticRegression(random_state = 0,
                              max_iter = 10000)

# Logit Model
LOGIT = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', logit_est)])


# Grid search by cross-validation
# ==============================================================================
grid = GridSearchCV(LOGIT,
        param_grid = param_grid,
        scoring    = 'f1_weighted',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

# Recover the best model
grid.fit(X_train, sett_train)
model_sett = grid.best_estimator_
pr_sett = model_sett.predict_proba(X_test)[:,1]

grid.fit(X_train, exp_train)
model_exp = grid.best_estimator_
pr_exp = model_exp.predict_proba(X_test)[:,1]

grid.fit(X_train, drp_train)
model_drp = grid.best_estimator_
pr_drp = model_drp.predict_proba(X_test)[:,1]


C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.32145899        nan        nan 0.32184076        nan
        nan 0.32145899        nan        nan 0.32184076        nan
        nan 0.32184076        nan        nan 0.32184076        nan
        nan 0.32184076        nan]
  warnings.warn(
C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the train scores are non-finite: [       nan 0.31984896        nan        nan 0.32074565        nan
        nan 0.32004558        nan        nan 0.32074565        nan
        nan 0.32074565        nan        nan 0.32074565        nan
        nan 0.32074565        nan]
  warnings.warn(
C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.88428706        nan        nan 0.88386986        n

In [8]:
from sklearn.ensemble import RandomForestClassifier
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__n_estimators': [150, 200],
              'model__max_features': [5, 7, 9],
              'model__max_depth'   : [None, 3, 10, 20]
             }



RFC = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', RandomForestClassifier(random_state=123))])


# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = RFC,
        param_grid = param_grid,
        scoring    = 'f1_weighted',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

# Recover the best model
grid.fit(X, cr_win)
model_cr_win = grid.best_estimator_
pr_cr_win = model_cr_win.predict_proba(X)[:,1]

grid.fit(X, cr_los)
model_cr_los = grid.best_estimator_
pr_cr_los = model_cr_los.predict_proba(X)[:,1]

grid.fit(X_3, cr_win_ph2)
model_cr_win_ph2 = grid.best_estimator_
pr_cr_win_ph2 = model_cr_win_ph2.predict_proba(X_3)[:,1]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {'model__n_estimators': [100, 200, 500],
              'model__max_features': ["auto", "sqrt", "log2"],
              'model__max_depth'   : [None, 3, 10, 20]
             }

# Random Forests
RFX = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', RandomForestRegressor(random_state=123))])

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = RFX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

# Recover the best model
grid.fit(X_1, tot_comp_1l)
model_tot_comp_1lrf = grid.best_estimator_
pr_tot_comp_1lrf = model_tot_comp_1lrf.predict(X_1)

In [15]:
grid.fit(X_2, dur_2l)
model_dur_2l = grid.best_estimator_
pr_dur_2l = model_dur_2l.predict(X_2)

Fitting 15 folds for each of 36 candidates, totalling 540 fits


In [16]:
from sklearn.ensemble import GradientBoostingRegressor

In [17]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__n_estimators'  : [500, 1000],
              'model__max_depth'     : [None, 1, 5, 10, 20],
              'model__subsample'     : [0.5, 1]
             }


GB  = GradientBoostingRegressor(
                        random_state        = 123,
                        # Activación de la parada temprana
                        validation_fraction = 0.1,
                        n_iter_no_change    = 5,
                        tol                 = 0.0001
                    )
# Gradient Boosting
GBX = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', GB)])


# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(GBX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

# Recover the best model
grid.fit(X_1, dur_1)
model_dur_1 = grid.best_estimator_
pr_dur_1 = model_dur_1.predict(X_1)

grid.fit(X_4, dur_4)
model_dur_4 = grid.best_estimator_
pr_dur_4 = model_dur_4.predict(X_4)

Fitting 15 folds for each of 20 candidates, totalling 300 fits
Fitting 15 folds for each of 20 candidates, totalling 300 fits


In [18]:
from sklearn.linear_model import Ridge
from scipy.stats import uniform

param_distributions = {
    'modelo__alpha': uniform(0.001, 10)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
RIDGEX = Pipeline([('preprocessing', preprocessor),
                 ('modelo', Ridge())])


# Búsqueda por validación cruzada
# ==============================================================================
grid = RandomizedSearchCV(
        estimator  = RIDGEX,
        param_distributions = param_distributions,
        n_iter     = 100,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )


grid.fit(X_31, dur_31)
model_dur_31 = grid.best_estimator_
pr_dur_31 = model_dur_31.predict(X_31)

Fitting 15 folds for each of 100 candidates, totalling 1500 fits


In [19]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

LRX = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', LinearRegression())])

model_tot_comp_3l = LRX.fit(X_3, tot_comp_3l)
pr_tot_comp_3l = model_tot_comp_3l.predict(X_3)

model_tot_comp_3 = LRX.fit(X_3, tot_comp_3)
pr_tot_comp_3 = model_tot_comp_3.predict(X_3)

model_dur_30 = LRX.fit(X_30, dur_30)
pr_dur_30 = model_dur_30.predict(X_30)

model_tot_comp_1l = LRX.fit(X_1, tot_comp_1l)
pr_tot_comp_1l = model_tot_comp_1l.predict(X_1)



In [68]:
dict_models = {
    'm1' : [model_sett, X, sett, pr_sett],
    'm2' : [model_cr_los, X, cr_los, pr_cr_los],
    'm3' : [model_cr_win, X, cr_win, pr_cr_win],
    'm4' : [model_exp, X, exp, pr_exp],
    'm5' : [model_drp, X, drp, pr_drp],
    'm6' : [model_cr_win_ph2, X_3, cr_win_ph2, pr_cr_win_ph2],
    'm7' : [model_tot_comp_1lrf, X_1, tot_comp_1l, pr_tot_comp_1lrf],
    'm8' : [model_tot_comp_3l, X_3, tot_comp_3l, pr_tot_comp_3l],
    'm9' : [model_dur_1, X_1, dur_1, pr_dur_1],
    'm10' : [model_dur_30, X_30, dur_30, pr_dur_30],
    'm11' : [model_dur_31, X_31, dur_31, pr_dur_31],
    'm12' : [model_dur_4, X_4, dur_4, pr_dur_4],
    'm13' : [model_dur_2l, X_2, dur_2l, pr_dur_2l],    
    'm14' : [model_tot_comp_1l, X_1, tot_comp_1l, pr_tot_comp_1l],
    'm15' : [model_tot_comp_3, X_3, tot_comp_3, pr_tot_comp_3]
}

df_imp = pd.DataFrame()
df_pre = pd.DataFrame()

In [89]:
dict_models_test = {
    'm1' : [model_sett, X, sett_test, pr_sett],
    'm4' : [model_exp, X, exp_test, pr_exp],
    'm5' : [model_drp, X, drp_test, pr_drp]
}


In [90]:
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

for model in dict_models_test :
    print(model)
    print('explained var:', explained_variance_score(dict_models_test[model][2], dict_models_test[model][3], multioutput='uniform_average'))
    print('r2:', r2_score(dict_models_test[model][2], dict_models_test[model][3], multioutput='variance_weighted'))
    cor = np.corrcoef(dict_models_test[model][2], dict_models_test[model][3])
    print('cor:',cor[0,1])
    fpr, tpr, thresholds = metrics.roc_curve(dict_models_test[model][2], dict_models_test[model][3], pos_label=1)
    print('auc:',metrics.auc(fpr, tpr))
    print('ps:',average_precision_score(dict_models_test[model][2], dict_models_test[model][3]))
    print('acc: ', accuracy_score(dict_models_test[model][2], np.round_(dict_models_test[model][3])))

m1
explained var: -8.299775272058163e-05
r2: -0.10370569028121635
cor: -0.0202604379643305
auc: 0.5227745144334948
ps: 0.6730253408267376
acc:  0.367
m4
explained var: -0.04811922214426523
r2: -0.049419355226975936
cor: 0.005493109804151553
auc: 0.5155475371063842
ps: 0.07976084538948741
acc:  0.927
m5
explained var: -0.021870834447731413
r2: -0.05030912793260556
cor: 0.058645580360483306
auc: 0.5433223542705986
ps: 0.22439551161329596
acc:  0.798


In [28]:
for model in dict_models :
    importancia = permutation_importance(
                estimator    = dict_models[model][0],
                X            = dict_models[model][1],
                y            = dict_models[model][2],
                n_repeats    = 100,
                n_jobs       = multiprocessing.cpu_count() - 1,
                random_state = 123
             )
    # Var importance
    df_importancia = pd.DataFrame(
                        {k: importancia[k] for k in ['importances_mean', 'importances_std']}
                     )
    name1 = 'imp_mean_'+model
    name2 = 'imp_std_'+model
    df_importancia.rename(columns={'importances_mean': name1}, inplace=True)
    df_importancia.rename(columns={'importances_std': name2}, inplace=True)
    df_imp = pd.concat([df_imp, df_importancia], axis=1)
    # BLP
    result = sm.OLS(dict_models[model][3], dict_models[model][1], missing='drop').fit()
    df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
    df = df[['coef','std err']]
    name1 = 'coef_'+model
    name2 = 'std_'+model
    df.rename(columns={'coef': name1}, inplace=True)
    df.rename(columns={'std err': name2}, inplace=True)    
    df_pre = pd.concat([df_pre, df], axis=1)
    

df_imp['feature'] = X.columns    
df_pre['feature'] = X.columns  

C:\Users\isaac\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\isaac\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [29]:
df_imp

,imp_mean_m1,imp_std_m1,imp_mean_m2,imp_std_m2,imp_mean_m3,imp_std_m3,imp_mean_m4,imp_std_m4,imp_mean_m5,imp_std_m5,...,imp_std_m11,imp_mean_m12,imp_std_m12,imp_mean_m13,imp_std_m13,imp_mean_m14,imp_std_m14,imp_mean_m15,imp_std_m15,feature
0,0.000000,0.000000,0.000168,0.000073,0.000000,0.000000,0.0,0.0,-0.000174,0.000164,...,0.007228,0.000012,0.000014,6.451071e-04,6.528729e-05,2.475295e-04,3.086653e-04,4.811013e-09,3.674616e-09,reclutamiento
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.055775e-03,1.239466e-03,1.340135e-08,4.306196e-08,giro_empresa_d2
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.878657e-04,2.538211e-04,0.000000e+00,0.000000e+00,giro_empresa_d3
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000148,0.000106,-5.160354e-07,2.727014e-08,5.719620e-04,5.482958e-04,0.000000e+00,0.000000e+00,giro_empresa_d4
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.005901,0.000000,0.000000,2.651994e-04,4.533158e-05,7.088560e-03,1.531704e-03,1.453881e-08,3.910036e-08,giro_empresa_d5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,-0.000004,0.000049,0.008798,0.000803,0.008832,0.000782,0.0,0.0,-0.000820,0.000311,...,1.138664,0.003661,0.000859,5.605584e-02,2.098369e-03,7.201903e+12,9.057450e+11,3.941126e+01,4.368918e+00,min_vac
68,-0.000004,0.000049,0.008661,0.000833,0.008455,0.000769,0.0,0.0,-0.000820,0.000311,...,1.138653,0.009878,0.001253,5.575939e-02,2.228573e-03,6.915468e+12,9.587071e+11,2.243375e+01,3.458619e+00,min_prima_vac
69,0.000010,0.000077,0.012625,0.001156,0.006110,0.000688,0.0,0.0,-0.000050,0.000405,...,0.039368,0.120333,0.008741,1.216532e-01,4.401728e-03,1.872672e+13,1.075159e+12,8.192314e+02,3.633275e+01,min_ley
70,0.000022,0.000135,0.033107,0.001381,0.010722,0.000821,0.0,0.0,-0.000058,0.000370,...,13.400978,1.860136,0.114284,1.744370e+00,6.106104e-02,6.877095e+10,5.332339e+10,-7.557792e-05,9.241929e-05,c_sal_caidos


In [30]:
df_pre

,coef_m1,std_m1,coef_m2,std_m2,coef_m3,std_m3,coef_m4,std_m4,coef_m5,std_m5,...,std_m11,coef_m12,std_m12,coef_m13,std_m13,coef_m14,std_m14,coef_m15,std_m15,feature
reclutamiento,2.200000e-03,2.000000e-03,-4.760000e-02,1.000000e-02,1.560000e-02,5.000000e-03,-6.000000e-04,1.000000e-03,-8.500000e-03,3.000000e-03,...,inf,-3.450000e-02,2.720000e-01,6.790000e-02,3.500000e-02,1.060000e-01,5.000000e-02,9.338000e+04,39500.000,reclutamiento
giro_empresa_d2,2.727000e-01,2.800000e-02,6.418000e-01,1.780000e-01,-3.020000e-02,8.900000e-02,4.790000e-02,1.000000e-02,2.044000e-01,5.400000e-02,...,inf,-1.512000e-06,6.750000e-07,-1.246000e-08,8.010000e-09,-6.083000e-08,8.400000e-08,-1.501000e+05,172000.000,giro_empresa_d2
giro_empresa_d3,-2.156000e-09,2.660000e-10,2.451000e-09,1.670000e-09,-2.418000e-10,8.360000e-10,7.979000e-11,9.250000e-11,-2.055000e-09,5.110000e-10,...,inf,5.141000e-07,5.070000e-07,-6.725000e-09,7.160000e-09,2.705000e-09,7.500000e-09,-4.820000e-02,0.018,giro_empresa_d3
giro_empresa_d4,2.560000e-01,1.500000e-02,3.200000e-03,9.400000e-02,-1.270000e-02,4.700000e-02,4.490000e-02,5.000000e-03,1.930000e-01,2.900000e-02,...,inf,-5.951000e-01,1.019000e+00,-1.372000e-08,7.280000e-09,6.128200e+00,4.430000e-01,-5.320000e-02,0.026,giro_empresa_d4
giro_empresa_d5,2.694000e-01,6.000000e-03,9.280000e-02,3.800000e-02,1.200000e-02,1.900000e-02,4.470000e-02,2.000000e-03,1.993000e-01,1.200000e-02,...,inf,1.767400e+00,7.450000e-01,-2.420000e-02,1.570000e-01,5.357400e+00,2.000000e-01,-9.869000e+04,91600.000,giro_empresa_d5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
min_vac,6.600000e-03,1.080000e-01,-1.973200e+00,6.790000e-01,-3.243000e-01,3.390000e-01,-3.250000e-01,3.700000e-02,1.086000e-01,2.070000e-01,...,inf,1.470750e+01,1.807400e+01,1.429700e+00,2.175000e+00,1.295620e+01,3.606000e+00,6.325000e+06,2270000.000,min_vac
min_prima_vac,1.606000e-01,1.960000e-01,-7.145000e-01,1.236000e+00,6.374000e-01,6.170000e-01,-5.200000e-02,6.800000e-02,-5.640000e-02,3.780000e-01,...,inf,1.614340e+01,2.748900e+01,-6.446400e+00,3.863000e+00,-5.038900e+00,6.091000e+00,-3.249000e+06,3580000.000,min_prima_vac
min_ley,-3.740000e-02,8.300000e-02,1.721500e+00,5.220000e-01,1.319000e-01,2.610000e-01,2.704000e-01,2.900000e-02,-7.560000e-02,1.590000e-01,...,inf,-1.499460e+01,1.478700e+01,1.456000e-01,1.565000e+00,-9.357200e+00,2.841000e+00,-4.410000e+06,1790000.000,min_ley
c_sal_caidos,7.618000e-09,5.270000e-09,3.707000e-07,3.320000e-08,1.080000e-07,1.660000e-08,-3.021000e-08,1.840000e-09,1.267000e-07,1.020000e-08,...,inf,1.440000e-05,1.820000e-06,3.211000e-06,1.920000e-07,6.195000e-07,2.010000e-07,-1.403000e-01,0.145,c_sal_caidos


In [31]:
import os
df_pre_imp = pd.merge(df_pre, df_imp)
df_pre_imp.to_csv(os.path.join(path + '/_aux/df_pre_imp.csv') , index = False)

In [32]:
df_pre_imp

,coef_m1,std_m1,coef_m2,std_m2,coef_m3,std_m3,coef_m4,std_m4,coef_m5,std_m5,...,imp_mean_m11,imp_std_m11,imp_mean_m12,imp_std_m12,imp_mean_m13,imp_std_m13,imp_mean_m14,imp_std_m14,imp_mean_m15,imp_std_m15
0,2.200000e-03,2.000000e-03,-4.760000e-02,1.000000e-02,1.560000e-02,5.000000e-03,-6.000000e-04,1.000000e-03,-8.500000e-03,3.000000e-03,...,8.544204e-03,0.007228,0.000012,0.000014,6.451071e-04,6.528729e-05,2.475295e-04,3.086653e-04,4.811013e-09,3.674616e-09
1,2.727000e-01,2.800000e-02,6.418000e-01,1.780000e-01,-3.020000e-02,8.900000e-02,4.790000e-02,1.000000e-02,2.044000e-01,5.400000e-02,...,-1.110223e-16,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.055775e-03,1.239466e-03,1.340135e-08,4.306196e-08
2,-2.156000e-09,2.660000e-10,2.451000e-09,1.670000e-09,-2.418000e-10,8.360000e-10,7.979000e-11,9.250000e-11,-2.055000e-09,5.110000e-10,...,-1.110223e-16,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.878657e-04,2.538211e-04,0.000000e+00,0.000000e+00
3,2.560000e-01,1.500000e-02,3.200000e-03,9.400000e-02,-1.270000e-02,4.700000e-02,4.490000e-02,5.000000e-03,1.930000e-01,2.900000e-02,...,-1.110223e-16,0.000000,0.000148,0.000106,-5.160354e-07,2.727014e-08,5.719620e-04,5.482958e-04,0.000000e+00,0.000000e+00
4,2.694000e-01,6.000000e-03,9.280000e-02,3.800000e-02,1.200000e-02,1.900000e-02,4.470000e-02,2.000000e-03,1.993000e-01,1.200000e-02,...,7.837018e-03,0.005901,0.000000,0.000000,2.651994e-04,4.533158e-05,7.088560e-03,1.531704e-03,1.453881e-08,3.910036e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,6.600000e-03,1.080000e-01,-1.973200e+00,6.790000e-01,-3.243000e-01,3.390000e-01,-3.250000e-01,3.700000e-02,1.086000e-01,2.070000e-01,...,6.146593e-01,1.138664,0.003661,0.000859,5.605584e-02,2.098369e-03,7.201903e+12,9.057450e+11,3.941126e+01,4.368918e+00
68,1.606000e-01,1.960000e-01,-7.145000e-01,1.236000e+00,6.374000e-01,6.170000e-01,-5.200000e-02,6.800000e-02,-5.640000e-02,3.780000e-01,...,6.146566e-01,1.138653,0.009878,0.001253,5.575939e-02,2.228573e-03,6.915468e+12,9.587071e+11,2.243375e+01,3.458619e+00
69,-3.740000e-02,8.300000e-02,1.721500e+00,5.220000e-01,1.319000e-01,2.610000e-01,2.704000e-01,2.900000e-02,-7.560000e-02,1.590000e-01,...,1.336367e-01,0.039368,0.120333,0.008741,1.216532e-01,4.401728e-03,1.872672e+13,1.075159e+12,8.192314e+02,3.633275e+01
70,7.618000e-09,5.270000e-09,3.707000e-07,3.320000e-08,1.080000e-07,1.660000e-08,-3.021000e-08,1.840000e-09,1.267000e-07,1.020000e-08,...,5.904663e+00,13.400978,1.860136,0.114284,1.744370e+00,6.106104e-02,6.877095e+10,5.332339e+10,-7.557792e-05,9.241929e-05


In [33]:
dill.dump_session('ref_reports.db')